In [1]:
print(p_year)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
url_src = "data/rfc_sample2.csv"
df = pd.read_csv(url_src, parse_dates=True)
url_src = "data/rfc_model2.csv"
model = pd.read_csv(url_src, parse_dates=True)
manufacturer = df['module_manufacturer'].value_counts().reset_index()
manufacturer2 = model['module_manufacturer'].value_counts().reset_index()
mod = df['module_model'].value_counts().reset_index()
mod2 = model['module_model'].value_counts().reset_index()
manufacturer.columns = ['manufacturer', 'amount']
manufacturer2.columns = ['number', 'amount']
manufacturer['number'] = manufacturer2['number']
mod.columns = ['model', 'amount']
mod2.columns = ['number', 'amount']
mod['number'] = mod2['number']
unit_name = ['module_manufacturer','module_model']
df = df.drop(['Unnamed: 0'], axis=1)
model = model.drop(['Unnamed: 0'], axis=1)
pred = model.loc[model['year'] >= p_year]
pred = pred.drop(['stars'], axis=1)
print('Prediction set legth:',len(pred))
t = t_size
r = r_state
n = n_est
model = model.loc[model['year'] < p_year]
ucol = ['total_installed_price','system_size_DC', 'module_manufacturer']
X = model[ucol]
y = model['stars']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = t, random_state= r)
clf = RandomForestClassifier(n_estimators = n) 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
ar_unique, i = np.unique(y_pred, return_counts=True)

# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 5), '%.')
predictions = clf.predict(pred[ucol])

# Assing stars
pred['stars'] = predictions

# Link stars to module
best2 = pred['module_model'].loc[pred.stars == pred.stars.max()].unique()
b2_df = pd.DataFrame(columns=['model','amount','number'])

for b in best2:
    b2_df = pd.concat([b2_df, mod.loc[mod['number'] == b]], axis=0)

    b2_df = b2_df.sort_values(by='amount',ascending=False).reset_index()
b2_df = b2_df.drop('index', axis=1)
link_models = df.loc[(df.year == 2019)&(df.module_model != 'Unknown')]
# Create filter list to models
filter_list = b2_df['model'].unique()
# Fecth data by filter list
best = link_models.loc[(link_models['module_model'].isin(filter_list))&(link_models.stars == 3)]
best = best.sort_values(by='p_s')
cols = ['module_manufacturer','module_model','built_in_meter_inverter','efficiency_module','p_s','system_size_DC','RES']
vcountcols = ['module_manufacturer','module_model','built_in_meter_inverter',
                     'efficiency_module','p_s','system_size_DC','RES','count']
mean = best.system_size_DC.mean()
# Funktio jolla haetaan haluttu kokoluokka ja hintakokosuhde
def get_res(x,psfloor,psceiling,):
    if x == 'a':
        r = best[cols].loc[(best.p_s > int(psfloor))&(best.p_s < int(psceiling))]
        r = r.value_counts().to_frame().reset_index()
        r.columns = vcountcols
        return r
    if x == 'xs':
        r = best[cols].loc[(best.p_s > int(psfloor))&(best.p_s < int(psceiling))&(best.system_size_DC < mean)]
        r = r.value_counts().to_frame().reset_index()
        r.columns = vcountcols
        return r.loc[r.system_size_DC < r.system_size_DC.mean()]
    if x == 's':
        r = best[cols].loc[(best.p_s > int(psfloor))&(best.p_s < int(psceiling))&(best.system_size_DC < mean)]
        r = r.value_counts().to_frame().reset_index()
        r.columns = vcountcols
        return r.loc[r.system_size_DC > r.system_size_DC.mean()]
    if x == 'l':
        r = best[cols].loc[(best.p_s > int(psfloor))&(best.p_s < int(psceiling))&(best.system_size_DC > mean)]
        r = r.value_counts().to_frame().reset_index()
        r.columns = vcountcols
        return r.loc[r.system_size_DC < r.system_size_DC.mean()]
    if x == 'xl':
        r = best[cols].loc[(best.p_s > int(psfloor))&(best.p_s < int(psceiling))&(best.system_size_DC > mean)]
        r = r.value_counts().to_frame().reset_index()
        r.columns = vcountcols
        return r.loc[r.system_size_DC > r.system_size_DC.mean()]
    
# Funktio kutsu
s = size
ra = floor + rang
res = get_res(s,floor,ra)
# Pientä sorttia.. p_s = price / size
# Täällä parhaissa tuleekin mittarillisia
s = pd.Series([1, 2, 3, 4])
print('Best 4 sorted by price size relation')
display(res.sort_values(by=['p_s']).head(4).set_index(s))
print('Best 4 sorted by efficieny')
display(res.sort_values(by='efficiency_module', ascending=False).head(4).set_index(s))
print('Best 4 sorted by efficieny with built in meter inverter')
res2 = res.loc[res.built_in_meter_inverter == 1]
res2.sort_values(by='efficiency_module', ascending=False).head(4).set_index(s)